In [1]:
import uuid
import numpy as np

In [2]:
class Node:
    def __init__(self, tensor):
        self.tensor = tensor
        self.grad = None
        self.operation = None
        self.parent_node = None
        self.previous_nodes = None
        self.tag = str(uuid.uuid4())
    
    def __repr__(self):
        return 'Node: ' + self.tag
    
    def __str__(self):
        return 'Node: ' + self.tag

In [3]:
class Tensor:
    """
    Tensor
    """
    def __init__(self, numpy_array):
        if not isinstance(numpy_array, np.ndarray):
            raise ValueError("Must be initialized with a Numpy array")
        self.value = numpy_array
        self.nodes = list()
        self.operation = None  # type of operation done to create this node.
        self.node = None
        
    def backward(self):
        self._compute_grads(self.nodes[-1])
        
    def get_grad(self):
        return sum([n.grad for n in self.nodes])
    
    def _compute_grads(self, node):
        if node.grad is None:
            node.grad = np.full(node.tensor.value.shape, 1)
        
        if node.previous_nodes is None:
            return
        else:
            n1, n2 = node.previous_nodes
            
            if node.operation == 'add':
                n1.grad = np.copy(node.grad)
                n2.grad = np.copy(node.grad)
            elif node.operation == 'mul':
                n1.grad = np.copy(node.grad) * np.copy(n2.tensor.value)
                n2.grad = np.copy(node.grad) * np.copy(n1.tensor.value)
            else:
                raise ValueError("Invalid operation")
            # tensor.grad = None  # remove for saving memory
            self._compute_grads(n1)
            self._compute_grads(n2)
    
    def _add_tensor(self, other):
        """
        Tensor addition (core)
        """
        if isinstance(other, Tensor):
            # check shapes
            if not self.value.shape == other.value.shape:
                raise ValueError("Tensors must have the same shape. shape1: {}, shape2: {}".\
                                     format(str(self.value.shape), str(other.value.shape)))
            operation_result = self.value + other.value
            
            # tensor
            new_tensor = Tensor(operation_result)
            new_tensor.operation = 'add'
            
            # node
            if self.node is None:
                self_node = Node(self)
                self.node = self_node
            else:
                self_node = self.node
            other_node = Node(other)
            new_node = Node(new_tensor)
            new_node.operation = 'add'
            self_node.parent_node = new_node
            other_node.parent_node = new_node
            new_node.previous_nodes = [self_node, other_node]
            
            other.node = other_node
            new_tensor.node = new_node
            
            
            # couple tensor-node
            if self_node not in self.nodes:
                self.nodes.append(self_node)
            else:
                print(self.nodes)
                raise ValueError("Something is wrong. (self)")
                
            if other_node not in other.nodes:
                other.nodes.append(other_node)
            else:
                raise ValueError("Something is wrong. (other)")
            
            if new_node not in new_tensor.nodes:
                new_tensor.nodes.append(new_node)
            else:
                raise ValueError("Something is wrong. (new_node)")
            
            return new_tensor
        else:
            raise ValueError("Operands must be of `Tensor` type.")
    
    def _mul_tensor(self, other):
        """
        Tensor multiplication (core)
        """
        if isinstance(other, Tensor):
            # check shapes
            if not self.value.shape == other.value.shape:
                raise ValueError("Tensors must have the same shape. shape1: {}, shape2: {}".\
                                     format(str(self.value.shape), str(other.value.shape)))
            operation_result = self.value * other.value
            
            # tensor
            new_tensor = Tensor(operation_result)
            new_tensor.operation = 'mul'
            
            # node
            if self.node is None:
                self_node = Node(self)
                self.node = self_node
            else:
                self_node = self.node
            other_node = Node(other)
            new_node = Node(new_tensor)
            new_node.operation = 'mul'
            self_node.parent_node = new_node
            other_node.parent_node = new_node
            new_node.previous_nodes = [self_node, other_node]
            
            other.node = other_node
            new_tensor.node = new_node
            
            # couple tensor-node
            if self_node not in self.nodes:
                self.nodes.add(self_node)
            else:
                print(self.nodes)
                raise ValueError("Something is wrong. (self)")
                
            if other_node not in other.nodes:
                other.nodes.add(other_node)
            else:
                raise ValueError("Something is wrong. (other)")
            
            if new_node not in new_tensor.nodes:
                new_tensor.nodes.add(new_node)
            else:
                raise ValueError("Something is wrong. (new_node)")
            
            return new_tensor
        else:
            raise ValueError("Operands must be of `Tensor` type.")
            
    def _convert_other_to_tensor(self, other):
        """
        Convert 'other' to Tensor with proper shape for 
        proper operation.
        """
        # convert other to Tensor
        if isinstance(other, int) or isinstance(other, float):
            t = Tensor(np.full(self.value.shape, other))
        elif isinstance(other, np.ndarray):
            other = np.broadcast_to(other, self.value.shape)
            t = Tensor(other)
        elif isinstance(other, Tensor):
            t = other  # no need to do anything
        else:
            raise ValueError("Invalid type")
        return t
    
    def __add__(self, other):
        print('add is called')
        # convert other to tensor
        t = self._convert_other_to_tensor(other)
        # do tensor addition
        return self._add_tensor(t)
    
    def __radd__(self, other):
        print('radd is called')
        return self.__add__(other)
            
    def __mul__(self, other):
        # convert other to tensor
        t = self._convert_other_to_tensor(other)
        # do tensor multiplication
        return self._mul_tensor(t)
    
    def __rmul__(self, other):
        pass
        
    def __sub__(self, other):
        # convert other to tensor
        t = self._convert_other_to_tensor(other * (-1))
        # do tensor addition
        return self._add_tensor(t)
    
    def __str__(self):
        r = self.value.__repr__()
        r = r.replace('array', 'Tensor')
        return r
    
    def __repr__(self):
        r = self.value.__repr__()
        r = r.replace('array', 'Tensor')
        return r

In [4]:
arr1 = np.random.rand(2, 3)
# arr2 = np.random.rand(2, 3)

In [5]:
t1 = Tensor(arr1)
# t2 = Tensor(arr2)
print('t1:', t1)
# print('t2:', t2)

t1: Tensor([[0.23997036, 0.26714305, 0.26883173],
       [0.41876565, 0.01369344, 0.78602728]])


In [6]:
t5 = t1 + t1 + 2

add is called
add is called
[Node: d16654ea-9c72-4d7d-9722-fe787fb89f66]


ValueError: Something is wrong. (self)

In [7]:
t5.nodes

[Node: 29b58adc-10e8-45f0-8b9d-efd2de53a26b]

In [8]:
t1.nodes

[Node: b75196f0-c1c4-4a58-be71-82da94af0fb3,
 Node: fcb6033e-2c36-484c-b8cc-e2e3e9eafdc5]

In [10]:
t5.nodes[0].previous_nodes[0].previous_nodes

In [8]:
t5.backward()

In [9]:
t1.nodes

[Node: 04c124c5-b5fe-4659-9288-cde2d33bec6a,
 Node: a9ed190b-23fa-4639-8ada-cc69343f5802,
 Node: 633630fe-879c-4032-956f-0a255fe08758]

In [10]:
t1.nodes[2].previous_nodes

In [11]:
t5.nodes[0].previous_nodes[0].previous_nodes

In [24]:
[n.grad for n in t1.nodes]

[None, None, array([[1, 1, 1],
        [1, 1, 1]])]

In [25]:
[n for n in t1.nodes]

[Node: bee201f0-f8ac-4a79-987c-2870f4fbf992,
 Node: cc559bc2-5449-4e2b-9432-cc661cff3f53,
 Node: db6908c4-35be-44e7-8f71-ec569acfbff9]

In [10]:
t1.nodes

[Node: 36e81509-e986-46a2-bc94-4018b688353f,
 Node: c54c01a1-599c-43de-bf29-4fc489da0b95,
 Node: 487a8c07-71a4-405a-9fe3-b63d33456e28]

In [8]:
[n.grad for n in t1.nodes]

[None, None, array([[1, 1, 1],
        [1, 1, 1]])]

In [12]:
[n.operation for n in t1.nodes]

[None, None, None]

In [10]:
t1.nodes

[Node: a17fbe70-f02f-497d-b20a-dc9ea9690a5b,
 Node: caf038f8-b6b8-473d-87b4-57799f93b1e8,
 Node: 13ff6616-f09e-4583-97cc-edc58c5187b5]

In [19]:
t5.nodes[0].previous_nodes[1].operation

In [5]:
t10 = (t1 - 2) * (t1 - 2)

In [6]:
t10.backward()

In [7]:
t1.grad

array([[-2.99519256, -3.70566587, -3.57598042],
       [-2.62173808, -2.41002425, -2.09688913]])

In [11]:
(2 * t1) - 4

Tensor([[-1.49759628, -1.85283293, -1.78799021],
       [-1.31086904, -1.20501212, -1.04844457]])

In [14]:
t1 + 2

Tensor([[2.50240372, 2.14716707, 2.21200979],
       [2.68913096, 2.79498788, 2.95155543]])

In [47]:
t10 = (t1 + 2) / (t1 + 2)

TypeError: unsupported operand type(s) for /: 'Tensor' and 'Tensor'

In [44]:
t10.backward()

In [45]:
t1.grad

array([[5.52619492, 4.71267724, 5.95470321],
       [4.45144276, 4.1928202 , 5.00457934]])

In [46]:
2 * arr1 + 4

array([[5.52619492, 4.71267724, 5.95470321],
       [4.45144276, 4.1928202 , 5.00457934]])

In [6]:
t20.backward()

NameError: name 't20' is not defined

In [ ]:
t1.grad

In [6]:
import torch

In [7]:
t = torch.rand(2, 3)
t

tensor([[0.8264, 0.3591, 0.8097],
        [0.5179, 0.3105, 0.6348]])

In [9]:
a = 'txt_another'

In [11]:
a.replace('txt', 'some')

'some_another'